In [1]:
using System;
using System.IO;
using System.Text.Json;
using System.Text.RegularExpressions;

public class LocalizationTargetEntry
{
    public required string source { get; set; }
    public required string target { get; set; }
}

public class LocalizationData
{
    public required string source { get; set; }
    public required string target { get; set; }
    public required Dictionary<string, Dictionary<string, LocalizationTargetEntry>> files { get; set; }
}

var localizationData = JsonSerializer.Deserialize<LocalizationData>(File.ReadAllText(Path.Combine(Environment.CurrentDirectory, "rawdata/raw_localization.json")));
var uuidToTranslationText = new Dictionary<string, string>();

foreach (var (fileIndex, translationList) in localizationData.files)
{
    foreach (var translation in translationList)
        uuidToTranslationText.Add(translation.Key.ToUpper(), translation.Value.target);
}

Console.WriteLine($"Total translation files: {localizationData.files.Count}");
Console.WriteLine($"Total translations: {uuidToTranslationText.Count}");

//
List<(string UUID, string Text)> loadDialogueListFromFile(string filePath)
{
    if (!File.Exists(filePath))
        return null;

    var parsedLines = new List<(string, string)>();
    var pattern = @"\[ ""(.*?)"" \], \# ""(.*?)""";

    foreach (var line in File.ReadAllLines(filePath))
    {
        var matches = Regex.Matches(line.Trim(), pattern);

        if (!matches.Any())
            continue;

        var match = matches[0];
        parsedLines.Add((match.Groups[1].Value.ToUpper(), match.Groups[2].Value));
    }

    return parsedLines;
}

var rawModFileData = loadDialogueListFromFile(Path.Combine(Environment.CurrentDirectory, "mod_ShutUpAloy.ini"));
var modFileCollisionCheck = new HashSet<string>();

Console.WriteLine($"Blacklisted translations in mod file: {rawModFileData.Count}");

// Make sure all UUIDs in the mod file are unique
void checkForDuplicateUUIDs()
{
    foreach (var (uuid, translation) in rawModFileData)
    {
        if (modFileCollisionCheck.Contains(uuid))
            Console.WriteLine($"Mod file contains duplicate translation UUID {uuid}");

        modFileCollisionCheck.Add(uuid);
    }
}

checkForDuplicateUUIDs();

// Now perform a direct comparison between all localized sentences in the game and in the mod file. Any text matches with
// differing UUIDs should be added.
void checkForSimilarSentences()
{
    string cleanLine(string input)
    {
        input = input.ToLower();
        input = Regex.Replace(input, @"\<time.*?\>", "");   // Remove <time0.18> meta tags
        input = input.Replace("\\n", " ");                  // Newlines -> spaces
        return input.Trim();
    }

    var uuidToTranslationTextCache = uuidToTranslationText
        .Select(x => (UUID: x.Key, Text: cleanLine(x.Value)))
        .ToList();

    var rawModFileDataCache = rawModFileData
        .Select(x => (UUID: x.UUID, Text: cleanLine(x.Text)))
        .ToList();

    foreach (var (uuid, text) in uuidToTranslationTextCache)
    {
        if (!modFileCollisionCheck.Contains(uuid) && rawModFileDataCache.Any(x => x.Text.Equals(text)))
            Console.WriteLine($"Translation ({uuid}, \"{text}\") should be in the mod file but isn't");
    }
}

checkForSimilarSentences();

// Fix lowercase UUIDs
var fixedLines = new List<string>();
var pattern = @"\[ ""(.*?)"" \], \# ""(.*?)""";

foreach (var line in File.ReadAllLines(Path.Combine(Environment.CurrentDirectory, "mod_ShutUpAloy.ini")))
{
    var matches = Regex.Matches(line.Trim(), pattern);

    if (!matches.Any())
    {
        fixedLines.Add(line);
        continue;
    }

    var match = matches[0];
    fixedLines.Add(string.Format("    [ \"{0}\" ], # \"{1}\"", match.Groups[1].Value.ToUpper(), match.Groups[2].Value));
}

File.WriteAllLines(Path.Combine(Environment.CurrentDirectory, "mod_testShutUpAloy.ini"), fixedLines);

Total translation files: 4778
Total translations: 58898
Blacklisted translations in mod file: 1224
Translation (21670A28-6329-5D40-A98B-97959D32C261, "what is that?") should be in the mod file but isn't
Translation (2C6E9306-CB86-BB4E-875A-A7BF701768EB, "what is that?") should be in the mod file but isn't
Translation (3C5782FC-2CAE-3A4C-94D0-996E67588184, "what is that?") should be in the mod file but isn't
Translation (8B2493EC-368F-2740-A9BC-907F2D6D55EA, "what is that?") should be in the mod file but isn't
Translation (DF34DC04-C4FE-2340-B06A-5D69B2E8ED74, "what is that?") should be in the mod file but isn't
